In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta,datetime
from pandas import DataFrame
import time
import cvxpy as cp
import numpy as np
from ortools.linear_solver import pywraplp
# from impala.dbapi import connect
# from impala.util import as_pandas

In [3]:
def str_date2date(s,n):
    p = datetime.strptime(s,"%Y-%m-%d")+timedelta(days=n)
    return datetime.strftime(p,"%Y-%m-%d")

def pred15(saudi_order):
    '''
    @parms:
        saudi_order     
    @return：
        pred15          
    '''
    pivot_sale = saudi_order.pivot_table(columns='pay_date',index='sku_id',values='sale_num')
    pivot_sale.fillna(0,inplace=True)
    sortlist=[i for i in range(1,16)]
    sale_data=np.array(pivot_sale)
    #加权移动平均算出15天的预测销量
    for i in range(15):
#         print(sale_data[:,0:10].shape)
#         print(np.arange(1,11)[::-1].reshape(10,1).shape)
        sale_num_pred= np.dot(sale_data[:,0:9],np.arange(1,10)[::-1].reshape(9,1))/45.

        sale_data=np.concatenate([sale_num_pred,sale_data],axis=1)
    pred_data=sale_data[:,0:15]
    #得到这批sku滚动30天的销量预测
    pred_sale_data=pred_data[:,::-1]
    pred15 = pd.DataFrame(pred_sale_data,columns=sortlist,index=pivot_sale.index)
    pred15.fillna(0,inplace=True)
    pred15['pred15']=np.round(np.dot(pred15.values,0.95**np.arange(1,pred15.shape[1]+1).reshape(-1,1)))
    pred15.reset_index(level=0,inplace=True)
    pred15 = pred15[['sku_id','pred15']]
    return pred15

In [4]:
import random
from random import choice
batch_sn_list=['A','B','C','D']
supp_code_list=['Tx','Ty','Th']
underwriting_ratio_list=[0.7,0.5,0.6]
gmt_created_list=[]
sku_id_list=['sku1','sku2','sku3','sku3','sku4','sku5','sku6','sku7','sku8','sku9','sku10']
goods_id=['gd','','','','','','']
in_price_list=[1,1.5,2,2.5,3,3.5,4,4.5,5,5.5]
total_num_list=[50,60,70,80,90]
sell_num_list=[34,40,43,45,50]

batch_sn=[]
supp_code=[]
underwriting_ratio=[]
sku_id=[]
in_price=[]
total_num=[]
sell_num=[]

for i in range(100):
    batch_sn.append(choice(batch_sn_list))
    supp_code.append(choice(supp_code_list))
    underwriting_ratio.append(choice(underwriting_ratio_list))
    sku_id.append(choice(sku_id_list))
    in_price.append(choice(in_price_list))
    total_num.append(choice(total_num_list))
    sell_num.append(choice(sell_num_list))

In [5]:
sku_gds_map={'sku1':'gds1','sku2':'gds1','sku3':'gds1','sku4':'gds2','sku5':'gds2','sku6':'gds2','sku7':'gds2','sku8':'gds3','sku9':'gds3','sku10':'gds3'}

In [6]:
sn_sku=pd.DataFrame({'batch_sn':batch_sn,
                     'supp_code':supp_code,
                     'rate':underwriting_ratio,
                     'sku_id':sku_id,
                     'in_price':in_price,
                     'stock_num':total_num,
                     'sold_num':sell_num,
                    })
sn_sku['goods_id']=sn_sku['sku_id'].map(sku_gds_map)
sn_sku['gmt_created']=1

In [7]:
sku_number_list=[1,2]
today_  = datetime.strftime(datetime.today(),"%Y-%m-%d")
yest    = str_date2date(today_,-1)

In [8]:
data_date_list=[]
for i in range(-50,-1):
    data_date_list.append(str_date2date(today_,i))

In [9]:
sku_id_s=[]
data_date=[]
sku_number=[]
for i in range(200):
    sku_id_s.append(choice(sku_id_list))
    data_date.append(choice(data_date_list))
    sku_number.append(choice(sku_number_list))

NameError: name 'saudi_order' is not defined

In [11]:
saudi_order=pd.DataFrame({'sku_id':sku_id_s,
                        'pay_date':data_date,
                        'sale_num':sku_number})
saudi_order['goods_id']=sn_sku['sku_id'].map(sku_gds_map).drop_duplicates()

In [12]:
def data_source(sn_sku,saudi_order,pred_start_date,clear_date):
    '''
    @buyout_data_path   买断数据源
    @sale_path          销售数据源
    
    '''
    sn_sku = sn_sku[sn_sku.stock_num>0]
    sn_sku.dropna(thresh=2,inplace=True)
    sn_sku['sn_left_num']=sn_sku['stock_num']-sn_sku['sold_num']
    # 订单数据
    saudi_order.dropna(thresh=2,inplace=True)
    saudi_order = saudi_order[saudi_order.sku_id.isin(list(set(sn_sku.sku_id)))]
    saudi_order = saudi_order[
                    (saudi_order["pay_date"] >= pred_start_date) & 
                    (saudi_order["pay_date"] <= clear_date)]
    return sn_sku,saudi_order

In [13]:
t1 = time.time()
today_  = datetime.strftime(datetime.today(),"%Y-%m-%d")
yest    = str_date2date(today_,-1)
n       = -10
pred_start_date=str_date2date(yest,-10)

In [14]:
sn_sku,saudi_order = data_source(sn_sku,saudi_order,pred_start_date,yest)
pred15_sale = pred15(saudi_order)
pred15_sale['pred15']=pred15_sale["pred15"].apply(lambda x:x*2)

In [15]:
# Determine whether the sales meet the buyout ratio, and screen the batches of the corresponding suppliers
buyout_notneed = sn_sku.groupby(by='batch_sn').apply(
        lambda x:
            pd.Series({
                    'stock_amt':sum(x['in_price']*x['stock_num']),
                    'sale_amt':sum(x['in_price']*x['sold_num']),
                    'sale_rate':sum(x['in_price']*x['sold_num'])/sum(x['in_price']*x['stock_num']),
                    'rate':np.mean(x['rate'])
                    })
        )

In [16]:
buyout_notneed_id =  buyout_notneed[
        buyout_notneed.rate<=
        buyout_notneed.sale_rate].index.values.tolist()

sn_sku = sn_sku[~sn_sku.batch_sn.isin(buyout_notneed_id)]
buyout_need = buyout_notneed.loc[
        buyout_notneed.rate>
        buyout_notneed.sale_rate,:]
buyout_need['need_buy_amt'] = np.array(buyout_need['stock_amt']*buyout_need['rate']-buyout_need['sale_amt'])
#buyout_need['stock_amt']*buyout_need['rate']-buyout_need['sale_amt']

# 买断
df_vars = sn_sku.merge(pred15_sale,on='sku_id',how='left')
df_vars.fillna(0,inplace=True)

df_b = buyout_need[['need_buy_amt']]

D:\ana\anaconda_insatll_path\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
df_b=df_b.reset_index()
df_b

,batch_sn,need_buy_amt
0,A,1.00
1,C,30.36


In [18]:
df_vars['buy_max']=df_vars['sn_left_num']-df_vars['pred15']
df_vars=df_vars[df_vars.buy_max>0]

In [19]:
result_list=[]
for sn in df_vars['batch_sn'].unique():
    B_sn=df_vars[df_vars.batch_sn==sn]
    min_amt=df_b[df_b.batch_sn==sn]['need_buy_amt'].values
    print(B_sn)
#     print(B_sn)
    n=len(B_sn)
    x = cp.Variable(n, integer=True)
    c=np.array(B_sn['in_price'])
    min_val=np.zeros(n)
    max_val=np.array(B_sn['buy_max'].tolist())
    A=np.eye(n, dtype=int)
    prob = cp.Problem(cp.Minimize(cp.sum(c.T@x)),
                 [cp.sum(c.T@x) >= min_amt,
                 A@x>=min_val,
                 A@x<=max_val])
    prob.solve()
    print(x.value)
    B_sn['final_buyout']=x.value
    print(prob.value)
    result_list.append(B_sn)
    print('................................................')
result_df=pd.concat(result_list)

   batch_sn supp_code  rate sku_id  in_price  stock_num  sold_num goods_id  \
0         C        Tx   0.7  sku10       4.5         60        40     gds3   
1         C        Th   0.7   sku3       5.0         60        45     gds1   
4         C        Ty   0.6   sku3       1.0         50        34     gds1   
13        C        Th   0.5   sku9       4.5         80        50     gds3   
14        C        Th   0.7   sku5       1.5         80        45     gds2   
15        C        Th   0.5   sku4       2.5         70        43     gds2   
18        C        Ty   0.7   sku1       1.0         60        40     gds1   
21        C        Ty   0.6   sku5       3.5         90        43     gds2   
22        C        Tx   0.7   sku3       4.5         70        40     gds1   
23        C        Ty   0.6   sku1       4.0         60        40     gds1   
24        C        Th   0.7   sku8       4.5         70        40     gds3   
26        C        Th   0.5   sku1       1.5         80        5

D:\ana\anaconda_insatll_path\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
  1.]
1.0
................................................


In [22]:
result_df[result_df.batch_sn=='C']

,batch_sn,supp_code,rate,sku_id,in_price,stock_num,sold_num,goods_id,gmt_created,sn_left_num,pred15,buy_max,final_buyout
0,C,Tx,0.7,sku10,4.5,60,40,gds3,1,20,14.0,6.0,6.0
1,C,Th,0.7,sku3,5.0,60,45,gds1,1,15,12.0,3.0,0.0
4,C,Ty,0.6,sku3,1.0,50,34,gds1,1,16,12.0,4.0,0.0
13,C,Th,0.5,sku9,4.5,80,50,gds3,1,30,2.0,28.0,-0.0
14,C,Th,0.7,sku5,1.5,80,45,gds2,1,35,14.0,21.0,-0.0
15,C,Th,0.5,sku4,2.5,70,43,gds2,1,27,24.0,3.0,0.0
18,C,Ty,0.7,sku1,1.0,60,40,gds1,1,20,6.0,14.0,-0.0
21,C,Ty,0.6,sku5,3.5,90,43,gds2,1,47,14.0,33.0,1.0
22,C,Tx,0.7,sku3,4.5,70,40,gds1,1,30,12.0,18.0,-0.0
23,C,Ty,0.6,sku1,4.0,60,40,gds1,1,20,6.0,14.0,-0.0
